In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

In [3]:
equity_domestic = ['Equity & Equity related']
arbitrage = ['Arbitrage','Arbitrage & Special Situations']
unlisted = ['(b) Unlisted']
equity_foreign = ['Equity & Equity related Foreign Investments']
derivatives = ['Derivatives']
money_market = ['Money Market Instruments']
options = ['Index / Stock Options']
futures = ['Index / Stock Futures']
end_note = ['Notes:']


In [17]:
table_extract_combos = [(equity_domestic, arbitrage),(arbitrage, unlisted), (equity_foreign, options), (options,money_market), (money_market, futures),(futures,end_note)]

In [18]:
col_list = ['instrument', 'isin','industry', 'quantity', 'value', 'percentage_of_net_assets','ytm']

In [4]:
def extract_clean_assets_from_fund_single_file(fund_name: str, file_path: Path, table_extract_combos:list):
    df = pd.read_excel(file_path)
    statement_period = extract_statement_period(df)
    df_dict_raw = extract_all_raw_tables(df, table_extract_combos)
    df_list_clean = clean_all_raw_tables(df_dict_raw)
    combined_df = pd.concat(df_list_clean).assign(statement_period=statement_period).reset_index(drop=True)
    return combined_df
    
    

In [15]:
def extract_statement_period(df):
    return df[df.iloc[:,1].astype('str').str.contains('Monthly Portfolio Statement')].iloc[:,1].values[0]

In [28]:
def get_idx_from_instrument_text(df, text, column=1):
    #return df[df.iloc[:, column]==text].index[0]
    try:
        idx = df[df.iloc[:, column].isin(text)].index[0]
    except Exception as e:
        print(f"Error: {e} for {text}")
        idx = None
    return idx

In [7]:
def get_raw_table(df, start_idx_text, end_idx_text):
    start_idx = get_idx_from_instrument_text(df, start_idx_text)
    end_idx = get_idx_from_instrument_text(df, end_idx_text)
    table_df = df.iloc[start_idx:end_idx-1, :]
    return table_df

In [31]:
get_raw_table(df, equity_domestic,end_note)

,Unnamed: 0,"Parag Parikh Flexi Cap Fund (An open-ended dynamic equity scheme investing across large cap, mid-cap, small-cap stocks)",Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
3,NaN,Equity & Equity related,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,(a) Listed / awaiting listing on Stock Exchanges,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,HDFC Bank Limited,INE040A01034,Banks,30292343,443040.66,0.0805,NaN,NaN
6,NaN,Bajaj Holdings & Investment Limited,INE118A01012,Finance,4596822,384583.92,0.0699,NaN,NaN
7,NaN,Power Grid Corporation of India Limited,INE752E01010,Power,119980027,311108.21,0.0565,NaN,NaN
...,...,...,...,...,...,...,...,...,...
152,NaN,Notes & Symbols :-,NaN,NaN,NaN,NaN,NaN,NaN,NaN
153,NaN,*Traded on US OTC Markets. Underlying shares a...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
154,NaN,^ The Name of the Industry is in accordance wi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
155,NaN,# The Name of the Industry is in accordance wi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
def extract_all_raw_tables(df, table_extract_combos):
    df_dict = {}
    for i,start, end in enumerate(table_extract_combos):
        try:
            df_dict[start[0]] = get_raw_table(df, start, end)
        except Exception as e:
            print(f'Error extracting table for {start} to {end} with error {e}')
    return df_dict

In [27]:
get_idx_from_instrument_text(df,options)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [9]:
def rename_columns(df, col_list):
    df.columns = col_list
    return df

In [10]:
def clean_raw_table(df):
    if len(df.iloc[:,1:-3].columns) != 7:
        df = df.assign(unknown = np.nan) # if only 6 columns, add a column of NaNs so that indexing below doesnt remove any useful column
    clean_df = (df
                .iloc[:,1:-3]
                .pipe(rename_columns, col_list)
                .query('(~quantity.isnull())|(~ytm.isnull())')
                .assign(
                        quantity = lambda x: x.quantity.astype(float),
                        value = lambda x: x.value.astype(float),
                        percentage_of_net_assets = lambda x: x.percentage_of_net_assets.astype('string').str.replace('$', '').str.replace('%', '').astype(float),
                        ytm = lambda x: x.ytm.astype(float)
                    )
                )
    return clean_df

In [11]:
def clean_all_raw_tables(df_dict):
    clean_df_list = []
    for key,df in df_dict.items():
        try:
            print(key)
            df = clean_raw_table(df).assign(table_name=key)
            clean_df_list.append(df)
        except Exception as e:
            print(e)
            continue
    return clean_df_list

In [14]:
def read_fund_files_path(fund_code: str) -> pd.DataFrame:
    """
    Process all Excel files for a given fund code.
    
    Args:
        fund_code (str): The fund code to process (e.g. 'PPFAS')
        
    Returns:
        pd.DataFrame: Combined processed data from all Excel files
    """
    try:
        # Get data directory path 
        data_dir = Path('data') / fund_code
        
        if not data_dir.exists():
            raise FileNotFoundError(f"Data directory not found: {data_dir}")
            
        # Get all Excel files
        excel_files = list(data_dir.glob('*.xls*'))
        
        if not excel_files:
            print(f"No Excel files found in {data_dir}")
            
        return excel_files
        
    except Exception as e:
        print(f"Error processing files: {e}")
        return []

In [147]:
comb_df_list = []
for f in file_paths:
    try:
        df = extract_clean_assets_from_fund_single_file('ppfas', f, table_extract_combos)
        comb_df_list.append(df)
        print(f'{f}- {df.percentage_of_net_assets.sum()}')
    except Exception as e:
        print(f'Error processing file {f} with error {e}')

Equity & Equity related
Arbitrage
Equity & Equity related Foreign Investments
Index / Stock Options
Money Market Instruments
Index / Stock Futures
could not convert string to float: '$0.00%'
data/ppfas/PPFCF_PPFAS_Monthly_Portfolio_Report_October_31_2024.xls- 1.0002
Equity & Equity related
Arbitrage
Equity & Equity related Foreign Investments
Index / Stock Options
Money Market Instruments
Index / Stock Futures
data/ppfas/PPFCF_PPFAS_Monthly_Portfolio_Report_September_30_2024.xls- 0.9937000000000001
Equity & Equity related
Arbitrage
Equity & Equity related Foreign Investments
Index / Stock Options
Money Market Instruments
Index / Stock Futures
data/ppfas/PPFCF_PPFAS_Monthly_Portfolio_Report_July_31_2024.xls- 0.9919000000000001
Equity & Equity related
Arbitrage
Equity & Equity related Foreign Investments
Index / Stock Options
Money Market Instruments
Index / Stock Futures
data/ppfas/PPFCF_PPFAS_Monthly_Portfolio_Report_August_31_2024.xls- 0.9962000000000001


In [148]:
combined_all_df = pd.concat(comb_df_list)

In [151]:
combined_all_df

,instrument,isin,industry,quantity,value,percentage_of_net_assets,ytm,table_name,statement_period
0,HDFC Bank Limited,INE040A01034,Banks,39684109.0,688797.08,0.0841,NaN,Equity & Equity related,"Monthly Portfolio Statement as on October 31, ..."
1,Power Grid Corporation of India Limited,INE752E01010,Power,180756561.0,579867.05,0.0708,NaN,Equity & Equity related,"Monthly Portfolio Statement as on October 31, ..."
2,Bajaj Holdings & Investment Limited,INE118A01012,Finance,5328201.0,546113.96,0.0667,NaN,Equity & Equity related,"Monthly Portfolio Statement as on October 31, ..."
3,Coal India Limited,INE522F01014,Consumable Fuels,117601373.0,531617.01,0.0649,NaN,Equity & Equity related,"Monthly Portfolio Statement as on October 31, ..."
4,ITC Limited,INE154A01025,Diversified FMCG,94519320.0,462010.44,0.0564,NaN,Equity & Equity related,"Monthly Portfolio Statement as on October 31, ..."
...,...,...,...,...,...,...,...,...,...
102,Jindal Steel & Power Limited September 2024 Fu...,NaN,Short,-8750.0,-85.27,0.0000,NaN,Index / Stock Futures,"Monthly Portfolio Statement as on August 31, 2024"
103,Bajaj Finserv Limited September 2024 Future,NaN,Short,-4500.0,-80.63,0.0000,NaN,Index / Stock Futures,"Monthly Portfolio Statement as on August 31, 2024"
104,Larsen & Toubro Limited September 2024 Future,NaN,Short,-1200.0,-44.74,0.0000,NaN,Index / Stock Futures,"Monthly Portfolio Statement as on August 31, 2024"
105,Tata Consultancy Services Limited September 20...,NaN,Short,-700.0,-32.06,0.0000,NaN,Index / Stock Futures,"Monthly Portfolio Statement as on August 31, 2024"


In [152]:
combined_all_df.groupby('statement_period').percentage_of_net_assets.sum()

statement_period
Monthly Portfolio Statement as on August 31, 2024       0.9962
Monthly Portfolio Statement as on July 31, 2024         0.9919
Monthly Portfolio Statement as on October 31, 2024      1.0002
Monthly Portfolio Statement as on September 30, 2024    0.9937
Name: percentage_of_net_assets, dtype: float64

In [143]:
check_df = extract_clean_assets_from_fund_single_file('ppfas', file_paths[2], table_extract_combos)

Equity & Equity related
Arbitrage
Equity & Equity related Foreign Investments
Index / Stock Options
Money Market Instruments
Index / Stock Futures


In [145]:
check_df.groupby('table_name').percentage_of_net_assets.sum()

table_name
Arbitrage                                      0.0072
Equity & Equity related                        0.7104
Equity & Equity related Foreign Investments    0.1284
Index / Stock Futures                         -0.0073
Index / Stock Options                          0.0002
Money Market Instruments                       0.1530
Name: percentage_of_net_assets, dtype: float64

In [144]:
check_df.percentage_of_net_assets.sum()

np.float64(0.9919000000000001)

In [ ]:
#### Test case

In [125]:
file_paths = read_fund_files_path('ppfas')

In [126]:
file_paths

[PosixPath('data/ppfas/PPFCF_PPFAS_Monthly_Portfolio_Report_May_31_2024 (1).xls'),
 PosixPath('data/ppfas/PPFCF_PPFAS_Monthly_Portfolio_Report_June_30_2024.xls'),
 PosixPath('data/ppfas/PPFCF_PPFAS_Monthly_Portfolio_Report_October_31_2024.xls'),
 PosixPath('data/ppfas/PPFCF_PPFAS_Monthly_Portfolio_Report_April_30_2024.xls'),
 PosixPath('data/ppfas/PPFCF_PPFAS_Monthly_Portfolio_Report_January_31_2024.xls'),
 PosixPath('data/ppfas/PPFCF_PPFAS_Monthly_Portfolio_Report_September_30_2024.xls'),
 PosixPath('data/ppfas/PPFCF_PPFAS_Monthly_Portfolio_Report_February_29_2024.xls'),
 PosixPath('data/ppfas/PPFCF_PPFAS_Monthly_Portfolio_Report_July_31_2024.xls'),
 PosixPath('data/ppfas/PPFCF_PPFAS_Monthly_Portfolio_Report_August_31_2024.xls'),
 PosixPath('data/ppfas/PPFCF_PPFAS_Monthly_Portfolio_Report_March_31_2024.xls')]

In [116]:
df = pd.read_excel(file_paths[2])

In [117]:
df

,Unnamed: 0,"Parag Parikh Flexi Cap Fund (An open-ended dynamic equity scheme investing across large cap, mid-cap, small-cap stocks)",Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,"Monthly Portfolio Statement as on October 31, ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,Name of the Instrument,ISIN,Industry / Rating,Quantity,Market/Fair Value\n (Rs. in Lakhs),% to Net\n Assets,YTM~,YTC^,NaN,NaN
3,NaN,Equity & Equity related,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,(a) Listed / awaiting listing on Stock Exchanges,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
304,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
305,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
306,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307,NaN,Tier I Benchmark’s Risk-o-meter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [118]:
statement_period = extract_statement_period(df)
statement_period

'Monthly Portfolio Statement as on October 31, 2024'

In [26]:
df2 = extract_all_raw_tables(df, table_extract_combos)

Error extracting table for ['Equity & Equity related Foreign Investments'] to ['Index / Stock Options'] with error index 0 is out of bounds for axis 0 with size 0
Error extracting table for ['Index / Stock Options'] to ['Money Market Instruments'] with error index 0 is out of bounds for axis 0 with size 0


### New Approach

In [119]:
raw_df = get_raw_table(df, equity_domestic,end_note)

In [120]:
raw2_df = (raw_df
 .assign(equity_domestic = lambda x: np.where(x.iloc[:,1].isin(equity_domestic), 'equity_domestic', ''),
         arbitrage = lambda x: np.where(x.iloc[:,1].isin(arbitrage), 'arbitrage', ''),
         unlisted = lambda x: np.where(x.iloc[:,1].isin(unlisted), 'unlisted', ''),
         equity_foreign = lambda x: np.where(x.iloc[:,1].isin(equity_foreign), 'equity_foreign', ''),
         options = lambda x: np.where(x.iloc[:,1].isin(options), 'options', ''),
         money_market = lambda x: np.where(x.iloc[:,1].isin(money_market), 'money_market', ''),
         futures = lambda x: np.where(x.iloc[:,1].isin(futures), 'futures', ''),
         section_1 = lambda x: (x.equity_domestic + x.arbitrage + x.unlisted + x.equity_foreign + x.options + x.money_market + x.futures),
         section = lambda x: x.section_1.replace('', np.nan).ffill(),
        )
 .drop(columns=['equity_domestic', 'arbitrage', 'unlisted', 'equity_foreign', 'options', 'money_market', 'futures', 'section_1'])
)

In [121]:
raw2_df

,Unnamed: 0,"Parag Parikh Flexi Cap Fund (An open-ended dynamic equity scheme investing across large cap, mid-cap, small-cap stocks)",Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,section
3,NaN,Equity & Equity related,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,equity_domestic
4,NaN,(a) Listed / awaiting listing on Stock Exchanges,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,equity_domestic
5,NaN,HDFC Bank Limited,INE040A01034,Banks,39684109,688797.08,0.0841,NaN,NaN,NaN,NaN,equity_domestic
6,NaN,Power Grid Corporation of India Limited,INE752E01010,Power,180756561,579867.05,0.0708,NaN,NaN,NaN,NaN,equity_domestic
7,NaN,Bajaj Holdings & Investment Limited,INE118A01012,Finance,5328201,546113.96,0.0667,NaN,NaN,NaN,NaN,equity_domestic
...,...,...,...,...,...,...,...,...,...,...,...,...
145,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,futures
146,NaN,Notes & Symbols :-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,futures
147,NaN,^ The Name of the Industry is in accordance wi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,futures
148,NaN,# The Name of the Industry is in accordance wi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,futures


In [84]:
def extract_full_raw_table(df):
    raw_df = (df
    .assign(equity_domestic = lambda x: np.where(x.iloc[:,1].isin(equity_domestic), 'equity_domestic', ''),
         arbitrage = lambda x: np.where(x.iloc[:,1].isin(arbitrage), 'arbitrage', ''),
         unlisted = lambda x: np.where(x.iloc[:,1].isin(unlisted), 'unlisted', ''),
         equity_foreign = lambda x: np.where(x.iloc[:,1].isin(equity_foreign), 'equity_foreign', ''),
         options = lambda x: np.where(x.iloc[:,1].isin(options), 'options', ''),
         money_market = lambda x: np.where(x.iloc[:,1].isin(money_market), 'money_market', ''),
         futures = lambda x: np.where(x.iloc[:,1].isin(futures), 'futures', ''),
         section_1 = lambda x: (x.equity_domestic + x.arbitrage + x.unlisted + x.equity_foreign + x.options + x.money_market + x.futures),
         section = lambda x: x.section_1.replace('', np.nan).ffill(),
        )
 .drop(columns=['equity_domestic', 'arbitrage', 'unlisted', 'equity_foreign', 'options', 'money_market', 'futures', 'section_1'])
)
    return raw_df
    
    

In [128]:
def clean_full_raw_table(df,statement_period):
    exclude_instruments = ["Name of the Instrument","Margin Fixed Deposit"]
    clean_df = (df
                .dropna(axis=1, how='all')
                .pipe(rename_columns, col_list+['section'])
                .query('(~quantity.isnull())|(~ytm.isnull())|(~industry.isnull())')
                .query('~(instrument.isin(@exclude_instruments))')
                .assign(
                        quantity = lambda x: x.quantity.astype(float),
                        value = lambda x: x.value.astype(float),
                        percentage_of_net_assets = lambda x: x.percentage_of_net_assets.astype('string').str.replace('$', '').str.replace('%', '').astype(float),
                        ytm = lambda x: x.ytm.astype(float),
                        statement_period = statement_period
                    )
                .reset_index(drop=True)
                )
    return clean_df

In [44]:
raw2_df.dropna(axis=1,how='all')

,"Parag Parikh Flexi Cap Fund (An open-ended dynamic equity scheme investing across large cap, mid-cap, small-cap stocks)",Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,section
3,Equity & Equity related,NaN,NaN,NaN,NaN,NaN,NaN,equity_domestic
4,(a) Listed / awaiting listing on Stock Exchanges,NaN,NaN,NaN,NaN,NaN,NaN,equity_domestic
5,HDFC Bank Limited,INE040A01034,Banks,30292343,443040.66,0.0805,NaN,equity_domestic
6,Bajaj Holdings & Investment Limited,INE118A01012,Finance,4596822,384583.92,0.0699,NaN,equity_domestic
7,Power Grid Corporation of India Limited,INE752E01010,Power,119980027,311108.21,0.0565,NaN,equity_domestic
...,...,...,...,...,...,...,...,...
152,Notes & Symbols :-,NaN,NaN,NaN,NaN,NaN,NaN,futures
153,*Traded on US OTC Markets. Underlying shares a...,NaN,NaN,NaN,NaN,NaN,NaN,futures
154,^ The Name of the Industry is in accordance wi...,NaN,NaN,NaN,NaN,NaN,NaN,futures
155,# The Name of the Industry is in accordance wi...,NaN,NaN,NaN,NaN,NaN,NaN,futures


In [122]:
clean1_df = clean_full_raw_table(raw2_df,statement_period)

In [123]:
clean1_df

,instrument,isin,industry,quantity,value,percentage_of_net_assets,ytm,section,statement_period
0,HDFC Bank Limited,INE040A01034,Banks,3.968411e+07,688797.080,0.0841,NaN,equity_domestic,"Monthly Portfolio Statement as on October 31, ..."
1,Power Grid Corporation of India Limited,INE752E01010,Power,1.807566e+08,579867.050,0.0708,NaN,equity_domestic,"Monthly Portfolio Statement as on October 31, ..."
2,Bajaj Holdings & Investment Limited,INE118A01012,Finance,5.328201e+06,546113.960,0.0667,NaN,equity_domestic,"Monthly Portfolio Statement as on October 31, ..."
3,Coal India Limited,INE522F01014,Consumable Fuels,1.176014e+08,531617.010,0.0649,NaN,equity_domestic,"Monthly Portfolio Statement as on October 31, ..."
4,ITC Limited,INE154A01025,Diversified FMCG,9.451932e+07,462010.440,0.0564,NaN,equity_domestic,"Monthly Portfolio Statement as on October 31, ..."
...,...,...,...,...,...,...,...,...,...
99,Bajaj Finserv Limited November 2024 Future,Short,-4500,-7.918000e+01,0.000,NaN,NaN,futures,"Monthly Portfolio Statement as on October 31, ..."
100,Hindustan Unilever Limited November 2024 Future,Short,-2700,-6.797000e+01,0.000,NaN,NaN,futures,"Monthly Portfolio Statement as on October 31, ..."
101,Larsen & Toubro Limited November 2024 Future,Short,-1200,-4.376000e+01,0.000,NaN,NaN,futures,"Monthly Portfolio Statement as on October 31, ..."
102,United Spirits Limited November 2024 Future,Short,-1400,-2.039000e+01,0.000,NaN,NaN,futures,"Monthly Portfolio Statement as on October 31, ..."


In [79]:
clean1_df.percentage_of_net_assets.sum()

np.float64(0.8902946122444025)

In [80]:
clean1_df.groupby('section').percentage_of_net_assets.sum()

section
arbitrage          0.007400
equity_domestic    0.718500
equity_foreign     0.150300
futures           -0.115405
money_market       0.129500
Name: percentage_of_net_assets, dtype: float64

In [131]:
clean_df_list = []
for f in file_paths:
    df = pd.read_excel(f)
    statement_period = extract_statement_period(df)
    raw_df = get_raw_table(df, equity_domestic,end_note)
    raw2_df = extract_full_raw_table(raw_df)
    clean_df = clean_full_raw_table(raw2_df,statement_period=statement_period)
    clean_df_list.append(clean_df)
    print(f'{f}- {clean_df.percentage_of_net_assets.sum()}')

data/ppfas/PPFCF_PPFAS_Monthly_Portfolio_Report_May_31_2024 (1).xls- 0.9914
data/ppfas/PPFCF_PPFAS_Monthly_Portfolio_Report_June_30_2024.xls- 0.992
data/ppfas/PPFCF_PPFAS_Monthly_Portfolio_Report_October_31_2024.xls- 0.9942000000000001
data/ppfas/PPFCF_PPFAS_Monthly_Portfolio_Report_April_30_2024.xls- 0.9943000000000001
data/ppfas/PPFCF_PPFAS_Monthly_Portfolio_Report_January_31_2024.xls- 0.8902946122444025
data/ppfas/PPFCF_PPFAS_Monthly_Portfolio_Report_September_30_2024.xls- 0.9937000000000001
data/ppfas/PPFCF_PPFAS_Monthly_Portfolio_Report_February_29_2024.xls- 0.891781780359427
data/ppfas/PPFCF_PPFAS_Monthly_Portfolio_Report_July_31_2024.xls- 0.9919000000000001
data/ppfas/PPFCF_PPFAS_Monthly_Portfolio_Report_August_31_2024.xls- 0.9962000000000001
data/ppfas/PPFCF_PPFAS_Monthly_Portfolio_Report_March_31_2024.xls- 0.8929858671666435


In [132]:
combined_df = pd.concat(clean_df_list)

In [134]:
combined_df.to_csv('output/raw_ppfas_portfolio.csv', index=False)